<a href="https://colab.research.google.com/github/mmosquera355/retoFapro/blob/main/desafioFaproColab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Author : M Angélica Mosquera Moreno
# Version: 1.0
# Date   : May 10, 2023
# Funcionalidad : el script lanzará una URL ngrok, ejemplo: https://dfe8-35-196-113-149.ngrok.io, para realizar la
#                 llamada a la API adicionar /consultarUF?fecha=2023-05-9, el formato de fecha es AÑO-MES-DIA
!pip install fastapi nest-asyncio pyngrok uvicorn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 357.3 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 681.2/681.2 kB 15.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.9/66.9 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 2.6 MB/s eta 0:00:00
  Created wheel for pyngrok: filename=pyngrok-6.0.0-py3-none-any.whl size=19867 sha256=8757f080b045f9c261e7874da5594a321038bd6779a65d21e44af8c3afbb6078
  Stored in directory: /root/.cache/pip/wheels/5c/42/78/0c3d438d7f5730451a25f7ac6cbf4391759d22a67576ed7c2c
Successfully built pyngrok


In [6]:

from fastapi import FastAPI
from fastapi.middleware.cors import CORSMiddleware
from fastapi import FastAPI
import pandas as pd
import datetime
from   datetime import date
import json
import nest_asyncio
from pyngrok import ngrok
import uvicorn
from fastapi import Response

app = FastAPI()

app.add_middleware(
    CORSMiddleware,
    allow_origins=['*'],
    allow_credentials=True,
    allow_methods=['*'],
    allow_headers=['*'],
)

def validar(valor):
  try:
	  entero = int(valor)
  except ValueError:
    return False #si la conversión falla devolvemos False

  return True


fechaMinima = date(int('2013'), int('01'), int('01'))
fechaMaxima = datetime.datetime.now()


@app.get("/")
async def root():
  URLexample = "/consultarUF?fecha=2023-5-9"
  return {'Hola! Para realizar la consulta deberas completar la URL abierta con el ejemplo al final del texto, recuerda que la fecha de esta en formato año-mes-dia ': URLexample}

@app.get("/consultarUF")
async def consultarUF(fecha: str):
  
  fechaCon = fecha.split("-")
  print(fechaCon)
  year     = fechaCon[0]
  month    = fechaCon[1]
  day      = fechaCon[2]

  validoYear = False
  validoYear = validar(year)
  
  if validoYear == True:
    if int(year) < 2013 or int(year) > fechaMaxima.year:
      validoYear = False

  validoMonth = False
  validoMonth = validar(month)
  if validoMonth == True:
    if int(month) < 1 or int(month) > 12:
      validoMonth = False

    if int(year) == fechaMaxima.year:
      if int(month) > fechaMaxima.month:
        validoMonth = False

  validoDay = False
  validoDay = validar(day)
  if validoDay == True:
    if int(month) == fechaMaxima.month:
      if int(day) > fechaMaxima.day:
        validoDay = False

  valido = True

  if validoDay is True and validoMonth is True and validoYear is True:
    try:
      fechaConsulta = date(int(year), int(month), int(day))
    except ValueError:
      valido = False
      response = "Ingresó una fecha invalida, por favor ingresar una fecha correcta: AÑO-MES-DIA"
  else:
    valido = False
    response = "Ingresó una fecha invalida, por favor ingresar una fecha correcta: AÑO-MES-DIA"

  #Consulta de tablas de la URl
  if valido is not False and validoDay is True and validoMonth is True and validoYear is True:
    urlBase = "http://www.sii.cl/valores_y_fechas/uf/uf"
    url     = urlBase + str(year) + ".htm"

    tablaUF = pd.read_html(url)

    UF = pd.DataFrame(tablaUF[len(tablaUF)-1])
    del(UF['Día'])
    UF.columns = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']

    print("El valor UF para la fecha de consulta es: " + UF[month][int(day)-1])
    print("Verificar en: " + url)
    #Respuesta JSON
    dic = {'fechaConsulta': fecha, 'ValorUF':UF[month][int(day)-1], 'url':url}
    response = json.dumps(dic, indent=4, sort_keys=True, default=str)
    print(response)

  return Response(content=response, media_type="application/json")



In [7]:
ngrok_tunnel = ngrok.connect(8000)
print('Public URL:', ngrok_tunnel.public_url)
nest_asyncio.apply()
uvicorn.run(app, port=8000)

Public URL: https://035a-104-155-215-56.ngrok.io


INFO:     Started server process [821]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     2803:1800:11c7:8000:6973:43f1:9bc7:ba94:0 - "GET / HTTP/1.1" 200 OK
['2023', '05', '05']
El valor UF para la fecha de consulta es: 35.903,96
Verificar en: http://www.sii.cl/valores_y_fechas/uf/uf2023.htm
{
    "ValorUF": "35.903,96",
    "fechaConsulta": "2023-05-05",
    "url": "http://www.sii.cl/valores_y_fechas/uf/uf2023.htm"
}
INFO:     2803:1800:11c7:8000:6973:43f1:9bc7:ba94:0 - "GET /consultarUF?fecha=2023-05-05 HTTP/1.1" 200 OK
INFO:     2803:1800:11c7:8000:6973:43f1:9bc7:ba94:0 - "GET /favicon.ico HTTP/1.1" 404 Not Found


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [821]
